In [2]:
%load_ext autoreload
%autoreload 2
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import argparse
import collections
import torch
import torch_geometric
import numpy as np
import data_loader.data_loaders as module_data
import model.loss as module_loss
import model.metric as module_metric
import model.model as module_arch
from parse_config import ConfigParser
# del Trainer
from trainer import Trainer
from utils import prepare_device
from model import model

array([0, 1, 2, 3, 4])

In [3]:
args = argparse.ArgumentParser(description='PyTorch Template')
args.add_argument('-c', '--config', default='config.json', type=str,
                  help='config file path (default: None)')
args.add_argument('-r', '--resume', default=None, type=str,
                  help='path to latest checkpoint (default: None)')
args.add_argument('-d', '--device', default=None, type=str,
                  help='indices of GPUs to enable (default: all)')
# args = args.parse_args(args=['--config', 'config.json',  '--device','2'])
# custom cli options to modify configuration from default values given in json file.
CustomArgs = collections.namedtuple('CustomArgs', 'flags type target')
options = [
    CustomArgs(['--lr', '--learning_rate'], type=float, target='optimizer;args;lr'),
    CustomArgs(['--bs', '--batch_size'], type=int, target='data_loader;args;batch_size')
]
config = ConfigParser.from_args(args, options)
print(type(config))

<class 'parse_config.ConfigParser'>


In [3]:
print(config.config["hparam"]["class_num"])
# print(metrics)

10


In [4]:
SEED = 123
torch_geometric.seed.seed_everything(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
# np.random.seed(SEED)

In [9]:
logger = config.get_logger('train')

# setup data_loader instances
print('Loading data...')
data_loader = config.init_obj('data_loader', module_data)
valid_data_loader = data_loader.split_validation()

Loading data...
torch.Size([18, 11721])
tensor([-0.7610, -0.1252, -0.6705, -0.6305, -0.4345,  1.0135,  0.1701, -0.1782,
         3.6215, -0.3276, -0.6090,  0.1315,  0.3671, -0.4615, -0.3648, -0.1866,
        -0.3048, -0.2494])
tensor([[-0.7610, -0.8069, -0.8232,  ..., -0.3436, -0.3436, -0.2357],
        [-0.1252, -0.0942, -0.1723,  ..., -0.3436,  2.7487, -0.2357],
        [-0.6705, -0.7052, -0.7331,  ..., -0.3436, -0.3436, -0.2357],
        ...,
        [-0.1866, -0.1998, -0.0867,  ..., -0.3436, -0.3436, -0.2357],
        [-0.3048, -0.3152, -0.2344,  ..., -0.3436, -0.3436, -0.2357],
        [-0.2494, -0.2404, -0.1685,  ..., -0.3436, -0.3436, -0.2357]])


In [5]:

# build model architecture, then print to console
device, device_ids = prepare_device(config['n_gpu'], config['device_id'])
# print(device)
print('Constructing model...')
model = config.init_obj('arch', module_arch, [data_loader.graphdata for i in range(18)])
logger.info(model)

# prepare for (multi-device) GPU training
# device, device_ids = prepare_device(config['n_gpu'], config['device_id'])

# device = "cpu"
model = model.to(device)
if len(device_ids) > 1:
    model = torch.nn.DataParallel(model, device_ids=device_ids)

# get function handles of loss and metrics
criterion = getattr(module_loss, config['loss'])
metrics = [getattr(module_metric, met) for met in config['metrics']]

# build optimizer, learning rate scheduler. delete every lines containing lr_scheduler for disabling scheduler
trainable_params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = config.init_obj('optimizer', torch.optim, trainable_params)
lr_scheduler = config.init_obj('lr_scheduler', torch.optim.lr_scheduler, optimizer)
# for state in optimizer.state.values():
# 	for k, v in state.items():
# 		if isinstance(v, torch.Tensor):
# 			state[k] = v.to(device)

Constructing model...
Skill_Evolve_Hetero(
  (criterion): NLLLoss()
  (dyskillhgnn): DySkillHGNN(
    (hgnn): HeteroHGNN(
      (heterognn): HeteroConv(num_relations=3)
    )
    (init_skill_emb): Embedding(11721, 128)
    (conv): ModuleDict(
      (nodeis_parentnode): normal_conv(
        (fc1): Linear(in_features=256, out_features=1, bias=True)
        (fc2): Linear(in_features=512, out_features=256, bias=True)
      )
      (nodeis_childnode): normal_conv(
        (fc1): Linear(in_features=256, out_features=1, bias=True)
        (fc2): Linear(in_features=512, out_features=256, bias=True)
      )
      (noderelatenode): normal_conv(
        (fc1): Linear(in_features=256, out_features=1, bias=True)
        (fc2): Linear(in_features=512, out_features=256, bias=True)
      )
    )
    (conv2): ModuleDict(
      (nodeis_parentnode): normal_conv(
        (fc1): Linear(in_features=256, out_features=1, bias=True)
        (fc2): Linear(in_features=512, out_features=256, bias=True)
      )
  

In [18]:
# import gc
# # del model 
# # del trainer
# gc.collect()


21

In [6]:

print("Start Training...")
# del trainer
trainer = Trainer(model, criterion, metrics, optimizer,
                  config=config,
                  device=device,
                  data_loader=data_loader,
                  valid_data_loader=valid_data_loader,
                  lr_scheduler=lr_scheduler)

trainer.train()

Start Training...
torch.Size([512, 128])
torch.Size([512, 128])
Train Epoch: 1 [0/88611 (0%)] Loss: 4.613319
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512, 

KeyboardInterrupt: 